<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/ADK_LLAMA3_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet

!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [3]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [5]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
    EarlyStoppingCallback
)

In [ ]:
torch.__version__

In [7]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.11.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Thu May  8 11:30:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   5

## Test Model and run Inference

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "/content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epochs1"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
nrec= randint(0, len(eval_dataset))
nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [10]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

'antral follicle count </s><s>[INST] the aim of this T0 was to evaluate the effect of a single dose of mg of the selective alpha adrenergic agonist clonidine on the cardiovascular and hormonal responses to a standardized mental arithmetic task in healthy volunteers the T0 was performed in a doubleblind randomized crossover design in healthy volunteers mean age years mean blood pressure mmhg mean heart rate bpm the subjects were studied on two occasions separated by a week interval on one occasion they received mg of clonidine and on the other occasion they received PL the order of the two treatments was randomized the subjects'

In [11]:
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")
print()
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
qc=str(ganswer).find('[INST]')
ganswer=ganswer[0:qc-7]
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0]
qc=str(ganswer).find('[/INST]')
if qc>0:
  ganswer=ganswer[qc+8:len(ganswer)]
print(f"Generated Answer:\n{ganswer}")
print()
if ganswer == oanswer:
  print("Match")
else:
  print("NO Match")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [12]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print()
print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [13]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
else:
    ganswer=ganswer[qc:len(ganswer)]
ganswer

'[/INST] antral follicle count </s><s>[INST] the aim of this T0 was to evaluate the effect of a single dose of mg of the selective alpha adrenergic agonist clonidine on the cardiovascular and hormonal responses to a standardized mental arithmetic task in healthy volunteers the T0 was performed in a doubleblind randomized crossover design in healthy volunteers mean age years mean blood pressure mmhg mean heart rate bpm the subjects were studied on two occasions separated by a week interval on one occasion they received mg of clonidine and on the other occasion they received PL the order of the two treatments was randomized the subjects'

In [14]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0-8]
qc=str(ganswer).find('[/INST]')

if qc>=0:
  ganswer=ganswer[qc+8:len(ganswer)]

print(f"Generated Answer:\n{ganswer}")

Generated Answer:
antral follicle count


In [ ]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from transformers import pipeline
from datasets import load_dataset

# Load a pre-trained sentence embedding model
model_embedding = SentenceTransformer('all-mpnet-base-v2')

# Load your test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

# Create the generation pipeline
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def evaluate(sample):
    prompt = sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.7,
                                  top_k=50, top_p=0.7, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer = outputs[0]['generated_text'][len(prompt):].strip()
    qc0 = str(ganswer).find('[/INST]')
    qc1 = str(ganswer).find('[INST]')
    ganswer = str(ganswer)[qc0 + 8:qc1 - 8]

    oanswer = str(sample['label'])
    oanswer = oanswer[2:len(oanswer) - 2]

    if len(oanswer) == 0:
        oanswer = 'Not possible to get or use'

    #print('\n\n')
    #print(f"Question: {prompt}")
    #print(f"Generated Answer: {ganswer}")
    #print(f"Original Answer: {oanswer}")

    # Generate sentence embeddings
    embedding1 = model_embedding.encode(ganswer, convert_to_tensor=True)
    embedding2 = model_embedding.encode(oanswer, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_sim = util.cos_sim(embedding1, embedding2).item()

    # Set a similarity threshold (adjust as needed)
    if cosine_sim >= 0.5:
        #print("Match")
        return 1
    else:
        return 0


In [ ]:
success_rate = []
number_of_eval_samples = 10  # Adjust the number of samples as needed

# Evaluate the model
for n in tqdm(range(number_of_eval_samples)):
    s = eval_dataset[n]
    success_rate.append(evaluate(s))

 70%|███████   | 7/10 [01:57<00:50, 16.73s/it]

In [16]:
# Calculate accuracy
accuracy = sum(success_rate) / len(success_rate)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5


## AGENT

In [ ]:
!pip install google-adk -q

In [ ]:
!pip install nest_asyncio -q

In [ ]:
import nest_asyncio
import asyncio
import os
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


# Define the local path to your fine-tuned model
# This path is from your Google Drive mount in the Colab environment
FINE_TUNED_MODEL_PATH = "/content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epochs1"

# Load the model and tokenizer from the local path using transformers
# We assume this code is run in an environment where this path is accessible
try:
    # Determine the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(FINE_TUNED_MODEL_PATH)
    # Set padding token if it's not already set
    if tokenizer.pad_token is None:
         tokenizer.pad_token = tokenizer.eos_token
         if tokenizer.pad_token is None:
             # Fallback if EOS token is also not set
             tokenizer.add_special_tokens({'pad_token': '[PAD]'})


    # Load the model
    # Use torch_dtype=torch.bfloat16 if your GPU supports it for potentially faster inference
    model = AutoModelForCausalLM.from_pretrained(FINE_TUNED_MODEL_PATH, torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32).to(device)
    model.eval() # Set the model to evaluation mode

    print(f"Successfully loaded model and tokenizer from {FINE_TUNED_MODEL_PATH}")

except Exception as e:
    print(f"Error loading model or tokenizer from {FINE_TUNED_MODEL_PATH}. Please ensure the path is correct and the necessary libraries are installed.")
    print(f"Details: {e}")
    # Exit or handle the error appropriately if the model cannot be loaded
    model = None
    tokenizer = None


# Define a simple agent class inheriting from ADK's Agent
class LocalFineTunedMedicalAgent(Agent):
    def __init__(self, name: str, instruction: str, model, tokenizer):
        # Initialize the Agent with name and instruction, model is a string
        # Set model path instead of None as ADK expects a string
        super().__init__(name=name, instruction=instruction, model=FINE_TUNED_MODEL_PATH)
        self._local_model = model
        self._local_tokenizer = tokenizer
        self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # The core asynchronous method where the agent processes input
    async def _run_async_impl(self, context): # Remove type hint for InvocationContext
        # Access the user's query from the invocation context
        user_query = context.request.message.text

        # print(f"Agent '{self.name}' received query: {user_query}") # Uncomment for more detailed logging

        # --- Text Generation using the locally loaded transformers model ---
        try:
            # Prepare the prompt
            # You might want to format the prompt based on your model's
            # preferred input format (e.g., chat format like Llama 3)
            # For this example, we'll use a simple instruction format derived from your notebook
            # Assuming your model was fine-tuned with prompts like "[INST] instruction [/INST]"
            prompt = f"[INST] {self.instruction}\n{user_query} [/INST]"


            # Tokenize the input
            inputs = self._local_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(self._device)

            # Generate text
            with torch.no_grad(): # Disable gradient calculation for inference
                 outputs = self._local_model.generate(
                     **inputs,
                     max_new_tokens=256, # Limit the generated response length
                     num_return_sequences=1,
                     do_sample=True,
                     top_k=50,
                     top_p=0.95,
                     temperature=0.7,
                     # Add other generation parameters as needed
                     pad_token_id=self._local_tokenizer.pad_token_id # Ensure pad_token_id is set for generation
                 )

            # Decode the generated tokens
            generated_text_with_prompt = self._local_tokenizer.decode(outputs[0], skip_special_tokens=False)

            # Post-process the generated text to extract the agent's response
            # Based on the prompt format "[INST] ... [/INST] response", we want the part after [/INST]
            if "[/INST]" in generated_text_with_prompt:
                generated_text = generated_text_with_prompt.split("[/INST]", 1)[1].strip()
            else:
                 # Fallback if the expected format is not present
                 generated_text = generated_text_with_prompt.strip()

            # Further cleaning might be needed depending on model output (e.g., cutting off at a specific token or phrase)
            # If the model generates a new turn of the prompt, cut it off
            if "[INST]" in generated_text:
                 generated_text = generated_text.split("[INST]", 1)[0].strip()


        except Exception as e:
            generated_text = f"An error occurred during text generation: {e}"
            print(f"Error during text generation with transformers: {e}")

        # Yield the final response from the agent
        yield context.AgentResponse( # Use context.AgentResponse directly
            message=context.AgentMessage(text=generated_text) # Use context.AgentMessage directly
        )


Using device: cuda


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Successfully loaded model and tokenizer from /content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epochs1


In [ ]:
!pip show google-adk

Name: google-adk
Version: 0.4.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-secret-manager, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-trace, opentelemetry-sdk, pydantic, python-dotenv, PyYAML, sqlalchemy, tzlocal, uvicorn
Required-by: 


In [ ]:
from google.adk import AgentMessage #Import the AgentMessage class here


In [ ]:
async def main():
    # Ensure model and tokenizer were loaded successfully
    if model is None or tokenizer is None:
        print("Model loading failed. Exiting.")
        return

    # 1. Instantiate the in-memory services for artifacts and sessions
    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    # 2. Create an instance of your local fine-tuned agent
    agent_name = "LocalFineTunedMedicalAssistant"
    agent_instruction = (
        "You are a highly specialized AI medical assistant, fine-tuned to provide information "
        "based on medical literature. Answer questions accurately and concisely, focusing on "
        "information related to medical concepts and terminology."
    )

    medical_agent = LocalFineTunedMedicalAgent(
        name=agent_name,
        instruction=agent_instruction,
        model=model,          # Pass the loaded transformers model
        tokenizer=tokenizer   # Pass the loaded transformers tokenizer
    )

    # 3. Create a Runner instance
    runner = Runner(
        agent=medical_agent,
        session_service=session_service,
        artifact_service=artifact_service,
        app_name="MyMedicalApp"  # Add app_name here with a suitable name
    )

    # 4. Define a test question
    test_question = """while diminished ovarian reserve dor predicts decreased ovarian response to
stimulation it does not necessarily foretell about the fecundity cycle acco
rding to bolognas criteria laid down by the european society of human repro
duction and embryology old age abnormal ovarian reserve tests such as AFC a
fc and antimullerian hormone amh as well as prior suboptimal response to st
imulation are the main AF representing dor unfavorable response to maximal
stimulation on two previous occasions may also represent dor among the ovar
ian reserve tests amh and afc are the most predictive values for dor AF whi
ch may give rise to dor include environmental factors autoimmune or metabol
ic disorders infections genetic abnormalities and iatrogenic causes such as
smoking chemotherapy radiation and gynecologic surgeries besides studies ha
ve proposed endometriosis as a key contributor to dor and hence emphasized
on its proper management to prevent additional damages leading to compromis
ed fertility in summary dor is found to be a clinical challenge in the prac
tice of fertility care with controversial countermeasures to prevent or tre
at the condition nevertheless some promising measure such as oocyte embryo
and tissue cryopreservation ovarian transplantation dietary supplementation
and the transfer of mitochondria have offered hopes towards ameliorating th
e burden of dor this review attempts to discuss dor from different perspect
ives and summarize some existing hopes in clinical practice"""

    test_original_answer = "antral follicle count" # The expected answer for comparison

    print('\n')
    print(f"\n--- Testing the Agent (Basic Runner Call) ---")
    print(f"Input Question: {test_question}")
    print('\n')
    print(f"Expected Answer: {test_original_answer}")
    print('\n')

    # Use a unique user and session ID for this test
    user_id = "test_user"
    session_id = "test_session"

    try:
        # Execute the agent with user_id and session_id only
        run_result = await runner.run(user_id=user_id, session_id=session_id)
        async for event in run_result:
            if event.is_user_query():
                # If it expects a response, send the test question
                await runner.respond(user_id=user_id, session_id=session_id, response=test_question)
            if event.is_final_response():
                print(f"Agent Generated Answer: {event.message.text}")

    except Exception as e:
        print(f"An error occurred during agent execution: {e}")

In [ ]:
# Entry point to run the asynchronous example
if __name__ == "__main__":
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())




--- Testing the Agent (Basic Runner Call) ---
Input Question: while diminished ovarian reserve dor predicts decreased ovarian response to
stimulation it does not necessarily foretell about the fecundity cycle acco
rding to bolognas criteria laid down by the european society of human repro
duction and embryology old age abnormal ovarian reserve tests such as AFC a
fc and antimullerian hormone amh as well as prior suboptimal response to st
imulation are the main AF representing dor unfavorable response to maximal
stimulation on two previous occasions may also represent dor among the ovar
ian reserve tests amh and afc are the most predictive values for dor AF whi
ch may give rise to dor include environmental factors autoimmune or metabol
ic disorders infections genetic abnormalities and iatrogenic causes such as
smoking chemotherapy radiation and gynecologic surgeries besides studies ha
ve proposed endometriosis as a key contributor to dor and hence emphasized
on its proper management 